In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jane-street-real-time-market-data-forecasting/responders.csv
/kaggle/input/jane-street-real-time-market-data-forecasting/sample_submission.csv
/kaggle/input/jane-street-real-time-market-data-forecasting/features.csv
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=4/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=5/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=6/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=3/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=1/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=8/part-0.parquet
/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=2/part-0.parquet
/kaggle/input/jane-street-real-time-market

In [2]:
import pandas as pd
import polars as pl
import gc

test_df = pl.scan_parquet(
    f"/kaggle/input/preprocessed_data_mini/tensorflow2/default/1/training_data.parquet"
).collect().to_pandas()

In [3]:
val_df = pl.scan_parquet(
    f"/kaggle/input/preprocessed_data_mini/tensorflow2/default/1/validation_data.parquet"
).collect().to_pandas()

# 데이터 처리

In [4]:
# 데이터 LSTM 입력화

def dat_to_input(data):
    features = data.filter(regex='^feature_')
    responders = data.filter(regex='^responder_')
    weights = data['weight']

    gc.collect()

    # Convert to numpy arrays for TensorFlow
    X = features.values  # Features for input

    # Assuming you have a DataFrame `y_train` with all responders <- noooo
    y = responders[['responder_6']].values  # Keep only responder_6

    # run if nan or inf exists - 
    #X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
    #y = np.nan_to_num(y, nan=0.0, posinf=0.0, neginf=0.0)

    return X,y


gc.collect()

0

In [5]:
X, y = dat_to_input(test_df)

In [6]:
X_val, y_val = dat_to_input(val_df)

In [7]:
X.shape

(8649459, 79)

In [8]:
X = X.reshape(-1, 3, 79)
X_val = X_val.reshape(-1,1,79)

print(X.shape, X_val.shape)

(2883153, 3, 79) (452056, 1, 79)


In [9]:
gc.collect()

0

# AutoEncoder

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, GRU, Input, TimeDistributed, Reshape

In [11]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Reshape
from keras.losses import Huber
from keras.optimizers import Adam

# 입력 레이어
input_layer = Input(shape=(X.shape[1], X.shape[2]))  # (타임스텝, 특성)

# 인코더: 다층 LSTM
encoded = LSTM(128, activation='relu', return_sequences=True)(input_layer)  # 첫 번째 LSTM
encoded = Dropout(0.2)(encoded)  # Dropout으로 과적합 방지
encoded = LSTM(64, activation='relu', return_sequences=False)(encoded)  # 두 번째 LSTM
encoded = BatchNormalization()(encoded)  # BatchNormalization으로 학습 안정화

# Bottleneck
bottleneck = Dense(64, activation='relu')(encoded)  # 잠재 공간
bottleneck = Dropout(0.2)(bottleneck)  # Dropout 추가

# 디코더: Dense -> LSTM
decoded = Dense(X.shape[1] * X.shape[2], activation='relu')(bottleneck)  # Latent 공간을 펼치기
decoded = Reshape((X.shape[1], X.shape[2]))(decoded)  # 원래 시계열 차원으로 복원
decoded = LSTM(64, activation='relu', return_sequences=True)(decoded)  # LSTM으로 시계열 복원
decoded = Dropout(0.2)(decoded)
decoded = LSTM(X.shape[2], activation='relu', return_sequences=True)(decoded)

# Autoencoder 모델
autoencoder = Model(inputs=input_layer, outputs=decoded)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss=Huber(delta=1.0))  # Huber Loss 적용
autoencoder.summary()

# 학습
autoencoder.fit(X, X, epochs=5, batch_size=64, validation_split=0.2, verbose=1)  # Epoch 조정 가능

# 인코더 모델
encoder = Model(inputs=input_layer, outputs=bottleneck)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 3, 79)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 3, 128)              │         106,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 3, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 237)                 │          15,405 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 3, 79)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 3, 64)               │          36,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 3, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 3, 79)               │          45,504 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 258,093 (1008.18 KB)

 Trainable params: 257,965 (1007.68 KB)

 Non-trainable params: 128 (512.00 B)

Epoch 1/5
36040/36040 ━━━━━━━━━━━━━━━━━━━━ 160s 4ms/step - loss: 0.3256 - val_loss: 0.2310
Epoch 2/5
36040/36040 ━━━━━━━━━━━━━━━━━━━━ 140s 4ms/step - loss: 0.2859 - val_loss: 0.2296
Epoch 3/5
36040/36040 ━━━━━━━━━━━━━━━━━━━━ 138s 4ms/step - loss: 0.2780 - val_loss: 0.2282
Epoch 4/5
36040/36040 ━━━━━━━━━━━━━━━━━━━━ 139s 4ms/step - loss: 0.2738 - val_loss: 0.2331
Epoch 5/5
36040/36040 ━━━━━━━━━━━━━━━━━━━━ 140s 4ms/step - loss: 0.2715 - val_loss: 0.2316


In [12]:
gc.collect()

752

In [13]:
'''
# Autoencoder 모델 구축
input_layer = Input(shape=(X.shape[1], X.shape[2]))  # (타임스텝, 특성)
encoded = LSTM(128, activation='relu', return_sequences=False)(input_layer)  # 인코더

bottleneck = Dense(64, activation='relu')(encoded)  # 반환점 - bottleneck

# 디코더: Latent 공간 -> 원래 시계열 차원 복원
decoded = Dense(X.shape[1] * X.shape[2], activation='relu')(bottleneck)  # 전체 차원 펼치기
decoded = Reshape((X.shape[1], X.shape[2]))(decoded)  # (타임스텝, 특성)으로 복원

autoencoder = Model(inputs=input_layer, outputs=decoded)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

# Autoencoder 학습
autoencoder.fit(X, X, epochs=5, batch_size=32, validation_split=0.2, verbose=1) # 여기를 조정해서 학습 시간 단축 가능 - 현재 20분 정도

# Encoder 모델 구축
encoder = Model(inputs=input_layer, outputs=bottleneck)
'''

"\n# Autoencoder 모델 구축\ninput_layer = Input(shape=(X.shape[1], X.shape[2]))  # (타임스텝, 특성)\nencoded = LSTM(128, activation='relu', return_sequences=False)(input_layer)  # 인코더\n\nbottleneck = Dense(64, activation='relu')(encoded)  # 반환점 - bottleneck\n\n# 디코더: Latent 공간 -> 원래 시계열 차원 복원\ndecoded = Dense(X.shape[1] * X.shape[2], activation='relu')(bottleneck)  # 전체 차원 펼치기\ndecoded = Reshape((X.shape[1], X.shape[2]))(decoded)  # (타임스텝, 특성)으로 복원\n\nautoencoder = Model(inputs=input_layer, outputs=decoded)\nautoencoder.compile(optimizer='adam', loss='mse')\nautoencoder.summary()\n\n# Autoencoder 학습\nautoencoder.fit(X, X, epochs=5, batch_size=32, validation_split=0.2, verbose=1) # 여기를 조정해서 학습 시간 단축 가능 - 현재 20분 정도\n\n# Encoder 모델 구축\nencoder = Model(inputs=input_layer, outputs=bottleneck)\n"

In [14]:
'''
# 3. GRU 모델 구축
# Autoencoder를 통해 Latent Representation 추출
latent_train = encoder.predict(X)
latent_test = encoder.predict(X_val)

# GRU를 활용한 예측
gru_model = Sequential([
    GRU(128, activation='relu', input_shape=(1, latent_train.shape[1]), return_sequences=False),
    Dense(64, activation='relu'),
    Dense(y.shape[1], activation='linear')  # 다중 특성 예측
])

gru_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
gru_model.summary()

# GRU 입력 차원 맞추기
latent_train = latent_train.reshape(latent_train.shape[0], 1, latent_train.shape[1])
latent_test = latent_test.reshape(latent_test.shape[0], 1, latent_test.shape[1])

# GRU 학습
gru_model.fit(latent_train, y, epochs=5, batch_size=32, validation_split=0.2, verbose=1)

# GRU 예측
predictions = gru_model.predict(latent_test) # 다 같은 답을 내놓는다. 뭔가 문제가 있음. 
'''

"\n# 3. GRU 모델 구축\n# Autoencoder를 통해 Latent Representation 추출\nlatent_train = encoder.predict(X)\nlatent_test = encoder.predict(X_val)\n\n# GRU를 활용한 예측\ngru_model = Sequential([\n    GRU(128, activation='relu', input_shape=(1, latent_train.shape[1]), return_sequences=False),\n    Dense(64, activation='relu'),\n    Dense(y.shape[1], activation='linear')  # 다중 특성 예측\n])\n\ngru_model.compile(optimizer='adam', loss='mse', metrics=['mae'])\ngru_model.summary()\n\n# GRU 입력 차원 맞추기\nlatent_train = latent_train.reshape(latent_train.shape[0], 1, latent_train.shape[1])\nlatent_test = latent_test.reshape(latent_test.shape[0], 1, latent_test.shape[1])\n\n# GRU 학습\ngru_model.fit(latent_train, y, epochs=5, batch_size=32, validation_split=0.2, verbose=1)\n\n# GRU 예측\npredictions = gru_model.predict(latent_test) # 다 같은 답을 내놓는다. 뭔가 문제가 있음. \n"

In [15]:
'''
latent_train = encoder.predict(X)

# 1. GRU 모델 정의 및 학습
# latent_train은 훈련 데이터의 특성 벡터, y_train은 목표 변수입니다.
gru_model = Sequential([
    GRU(128, activation='relu', input_shape=(1, latent_train.shape[1]), return_sequences=False),
    Dense(64, activation='relu'),
    Dense(latent_train.shape[1], activation='linear')  # y_train의 특성 수에 맞게
])

gru_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
gru_model.summary()
'''
# 아래 모델로 대체 가능? 아래 모델도 층이 그다지 깊어 보이지는 않는다...

"\nlatent_train = encoder.predict(X)\n\n# 1. GRU 모델 정의 및 학습\n# latent_train은 훈련 데이터의 특성 벡터, y_train은 목표 변수입니다.\ngru_model = Sequential([\n    GRU(128, activation='relu', input_shape=(1, latent_train.shape[1]), return_sequences=False),\n    Dense(64, activation='relu'),\n    Dense(latent_train.shape[1], activation='linear')  # y_train의 특성 수에 맞게\n])\n\ngru_model.compile(optimizer='adam', loss='mse', metrics=['mae'])\ngru_model.summary()\n"

In [16]:
from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout, BatchNormalization

latent_train = encoder.predict(X)

# 1. GRU 모델 정의
gru_model = Sequential([
    # 첫 번째 GRU 레이어: 더 많은 유닛 추가 및 정규화 적용
    GRU(256, activation='tanh', input_shape=(1, latent_train.shape[1]), return_sequences=True),
    BatchNormalization(),
    Dropout(0.2),  # 과적합 방지를 위한 드롭아웃

    # 두 번째 GRU 레이어: 추가 유닛 및 정규화
    GRU(128, activation='tanh', return_sequences=False),
    BatchNormalization(),
    Dropout(0.2),

    # 첫 번째 Dense 레이어: 더 많은 노드 추가 및 활성화 함수 사용
    Dense(128, activation='relu'),
    BatchNormalization(),

    # 출력 레이어: y_train의 특성 수에 맞는 선형 활성화 함수 사용
    Dense(latent_train.shape[1], activation='linear')
])

# 2. 컴파일 단계
gru_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 3. 모델 구조 요약
gru_model.summary()

90099/90099 ━━━━━━━━━━━━━━━━━━━━ 128s 1ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 1, 256)              │         247,296 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 1, 256)              │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 1, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 128)                 │         148,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │           8,256 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 422,336 (1.61 MB)

 Trainable params: 421,312 (1.61 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [17]:
# GRU 모델 훈련
# latent_train을 3D 배열로 변환 (배치 크기, 타임스텝, 특성)
latent_train_reshaped = latent_train.reshape(latent_train.shape[0]//3, 3, latent_train.shape[1]) # Timestep을 3개로 근데 예측이 39step이면 데이터 형태를 좀 바꿔야 할 것 같은데?? 
gru_model_trained = gru_model.fit(latent_train_reshaped, y, epochs=5, batch_size=64, validation_split=0.2, verbose=1)

Epoch 1/5
12014/12014 ━━━━━━━━━━━━━━━━━━━━ 100s 8ms/step - loss: 0.7045 - mae: 0.5621 - val_loss: 0.7655 - val_mae: 0.5958
Epoch 2/5
12014/12014 ━━━━━━━━━━━━━━━━━━━━ 95s 8ms/step - loss: 0.6782 - mae: 0.5472 - val_loss: 0.7660 - val_mae: 0.5951
Epoch 3/5
12014/12014 ━━━━━━━━━━━━━━━━━━━━ 96s 8ms/step - loss: 0.6787 - mae: 0.5480 - val_loss: 0.7653 - val_mae: 0.5952
Epoch 4/5
12014/12014 ━━━━━━━━━━━━━━━━━━━━ 97s 8ms/step - loss: 0.6790 - mae: 0.5482 - val_loss: 0.7670 - val_mae: 0.5962
Epoch 5/5
12014/12014 ━━━━━━━━━━━━━━━━━━━━ 96s 8ms/step - loss: 0.6789 - mae: 0.5476 - val_loss: 0.7658 - val_mae: 0.5957


In [18]:
gru_model_trained.history

# 모델 개선 방안 강구 

{'loss': [0.6860094666481018,
  0.680534839630127,
  0.6801891922950745,
  0.6798590421676636,
  0.6797130107879639],
 'mae': [0.5519184470176697,
  0.5478960871696472,
  0.5479459166526794,
  0.5479980111122131,
  0.5480139255523682],
 'val_loss': [0.7654647827148438,
  0.7660158276557922,
  0.7652653455734253,
  0.7669832706451416,
  0.7657775282859802],
 'val_mae': [0.5957881212234497,
  0.595084011554718,
  0.5952016711235046,
  0.5962190628051758,
  0.595697820186615]}

In [19]:
# 안 해도 되나?
'''
# 2. 미래 39단위 예측
n_steps = 39 #(예측할 단위)

# 첫 번째 입력을 latent_train에서 가져옵니다. (예: 첫 번째 샘플)
input_sequence = latent_train[0].reshape(1, 1, latent_train.shape[1])  # (1, 1, latent_train.shape[1])로 변환

# 예측 결과를 저장할 리스트
future_predictions = []

# 예측을 39번 반복해서 진행
for i in range(n_steps):
    # GRU 모델로 예측
    pred = gru_model.predict(input_sequence)
    
    # 예측된 값을 결과에 추가
    future_predictions.append(pred[0])  # pred[0]은 예측된 타겟 벡터 (배치 크기가 1이라서)

    # 예측된 값을 다시 input_sequence에 추가하여 다음 예측에 사용
    input_sequence = np.append(input_sequence[:, 1:, :], pred.reshape(1, 1, latent_train.shape[1]), axis=1)

# 결과를 numpy 배열로 변환
future_predictions = np.array(future_predictions)
'''

'\n# 2. 미래 39단위 예측\nn_steps = 39 #(예측할 단위)\n\n# 첫 번째 입력을 latent_train에서 가져옵니다. (예: 첫 번째 샘플)\ninput_sequence = latent_train[0].reshape(1, 1, latent_train.shape[1])  # (1, 1, latent_train.shape[1])로 변환\n\n# 예측 결과를 저장할 리스트\nfuture_predictions = []\n\n# 예측을 39번 반복해서 진행\nfor i in range(n_steps):\n    # GRU 모델로 예측\n    pred = gru_model.predict(input_sequence)\n    \n    # 예측된 값을 결과에 추가\n    future_predictions.append(pred[0])  # pred[0]은 예측된 타겟 벡터 (배치 크기가 1이라서)\n\n    # 예측된 값을 다시 input_sequence에 추가하여 다음 예측에 사용\n    input_sequence = np.append(input_sequence[:, 1:, :], pred.reshape(1, 1, latent_train.shape[1]), axis=1)\n\n# 결과를 numpy 배열로 변환\nfuture_predictions = np.array(future_predictions)\n'

In [20]:
'''
# 예측 결과를 데이터프레임 형태로 변환 (원본 y와 같은 형태로 출력)
predicted_df = pd.DataFrame(future_predictions, columns=[f"target_{i+1}" for i in range(future_predictions.shape[1])])

# 예측 결과 출력
print(predicted_df)
'''

'\n# 예측 결과를 데이터프레임 형태로 변환 (원본 y와 같은 형태로 출력)\npredicted_df = pd.DataFrame(future_predictions, columns=[f"target_{i+1}" for i in range(future_predictions.shape[1])])\n\n# 예측 결과 출력\nprint(predicted_df)\n'

# 개선 방안 강구
현 상황: 데이터가 제대로 학습하지 못하고 있음.
- latent 및 y를 정규화한다
- Adam의 학습률을 낮추거나 증가시킨다
- 노드 추가 혹은 감소
- 드롭아웃 추가

# RES Submit - 여기가 문제다!

In [21]:
# Submit ERROR?? - CFG

lags_ : pl.DataFrame | None = None


# Replace this function with your inference code.
# You can return either a Pandas or Polars dataframe, though Polars is recommended.
# Each batch of predictions (except the very first) must be returned within 10 minutes of the batch features being provided.
def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    # All the responders from the previous day are passed in at time_id == 0. We save them in a global variable for access at every time_id.
    # Use them as extra features, if you like.
    global lags_
    if lags is not None:
        lags_ = lags
    # 1. Select the required feature columns and convert to numpy array for Keras
    X_test = test.select(CFG.feature_columns).to_numpy()
    X_test = np.where(np.isnan(X_test), mean_values, X_test)
    X_test = (X_test - min_values) / max_min_diff
    # 2. Make predictions using the Keras model
    y_pred = gru_model_trained.predict(X_test, batch_size=4096)
    
    # 3. Prepare the DataFrame for output
    predictions = test.select('row_id').with_columns(
        pl.Series("responder_6", y_pred.flatten())
    )
    # The predict function must return a DataFrame
    assert isinstance(predictions, pl.DataFrame | pd.DataFrame)
    # with columns 'row_id', 'responer_6'
    assert predictions.columns == ['row_id', 'responder_6']
    # and as many rows as the test data.
    assert len(predictions) == len(test)

    return predictions

In [22]:
import os
import kaggle_evaluation.jane_street_inference_server

inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
            '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )

GatewayRuntimeError: (<GatewayRuntimeErrorType.SERVER_RAISED_EXCEPTION: 3>, "name 'CFG' is not defined")